In [7]:
#pip install Cmake

In [8]:
# pip install prophet

In [9]:
#pip install --upgrade holidays

In [10]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
from prophet import Prophet,plot

In [11]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))
    return mape

In [12]:
directory = os.getcwd()

In [13]:
folder = 'EDA'
filename = 'ProcessedData.csv'
write_path = os.path.join(directory,folder,filename)

In [14]:
df = pd.read_csv(os.path.join(write_path))

In [15]:
agg = {      
    'Sales Volumes in L15': 'sum',
    'Holiday Flag': 'max',
    'Holiday Desc': 'max',
    'LockDown Flag': 'max',
    'Fuel Price': 'mean',
    'Loyalty Flag': 'max',
    'Estimated Budget in ZAR': 'sum',
     'Marketing Flag': 'max',
    'Estimated Budget in USD': 'sum'
}
sales_data = df.groupby(['Material Number','date']).aggregate(agg).reset_index()
sales_data['date'] = pd.to_datetime(sales_data['date'], format='%Y-%m-%d')
sales_data.sort_values(by=['Material Number','date'], inplace=True)

In [16]:
sales_data['Fuel Price L1'] = sales_data.groupby(['Material Number'])['Fuel Price'].shift(1)
sales_data['Fuel Price PCT Change'] = ((sales_data['Fuel Price']-sales_data['Fuel Price L1'])/sales_data['Fuel Price L1'])*100
sales_data['Fuel Price PCT Change'].fillna(0, inplace = True)

In [17]:
#Reading the data
train_date = ['2023-11-30','2023-12-31','2024-01-31','2024-02-29']
test_date = ['2023-12-01','2024-01-01','2024-02-01','2024-03-01']
end_date = ['2023-12-31','2024-01-31','2024-02-29','2024-03-17']
periods = [31,31,29,17]
material_list = sales_data['Material Number'].unique()

In [18]:
#Building Baseline Prophet Model
predicted_df = pd.DataFrame()
for i in range(0,4):
    for x in material_list:
        train_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] <= train_date[i])]
        df = train_data[['date','Sales Volumes in L15']]
        df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)

        # Prophet model
        m = Prophet()
        m.fit(df)

        # forecasting
        future = m.make_future_dataframe(periods=periods[i])

        # Predictionns
        forecast = m.predict(future)
        pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        pred = pred[(pred['ds'] >= test_date[i]) & (pred['ds'] <= end_date[i])]
        test_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] >= test_date[i]) & (sales_data['date'] <= end_date[i])]
        pred.reset_index(inplace = True)
        test_data.reset_index(inplace = True)
        forecast = pd.concat([test_data, pred], axis=1)
        predicted_df = pd.concat([predicted_df,forecast], axis = 0)

C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\772223918.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)
16:29:02 - cmdstanpy - INFO - Chain [1] start processing
16:29:07 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\772223918.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)
16:29:08 - cmdstanpy - INFO - Chain [1] start processing
16:29:08 - cmdstanpy - INFO - Chain [1

In [20]:
folder = 'Modelling Output'
subfolder = 'PROPHET'
filename = 'predictions.csv'
write_path = os.path.join(directory,folder,subfolder,filename)
predicted_df.to_csv(write_path)

In [21]:
#Building Prophet Model w Holidays
predicted_df = pd.DataFrame()
for i in range(0,4):
    for x in material_list:
        train_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] <= train_date[i])]
        df = train_data[['date','Sales Volumes in L15']]
        df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)

        #Adding holidays
        holiday_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Holiday Flag'] == 1)]['date'].unique()
        lockdown_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['LockDown Flag'] == 1)]['date'].unique()
        loyalty_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Loyalty Flag'] == 1)]['date'].unique()
        Marketing_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Marketing Flag'] == 1)]['date'].unique()
    
        public_holidays = pd.DataFrame({
        'holiday': 'public holidays',
        'ds': holiday_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        lockdows = pd.DataFrame({
        'holiday': 'lockdowns',
        'ds': lockdown_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        loyalty = pd.DataFrame({
        'holiday': 'loyalty',
        'ds': loyalty_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        marketing = pd.DataFrame({
        'holiday': 'loyalty',
        'ds': Marketing_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        holidays = pd.concat((public_holidays, lockdows,loyalty,marketing))
    
        #Adding Regressors (Price)
        df['Fuel Price'] = train_data['Fuel Price']
        test_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] >= test_date[i]) & (sales_data['date'] <= end_date[i])]
        m = Prophet(holidays=holidays)
        #m.add_regressor('Fuel Price')
        m.fit(df)
        future = m.make_future_dataframe(periods=periods[i])
        future = future[(future['ds'] >= test_date[i]) & (future['ds'] <= end_date[i])]
        #future = pd.merge(future, test_data[['date','Fuel Price']], left_on = 'ds', right_on = 'date', how= 'left')
        #future.drop('date', inplace = True, axis = 1)
        forecast = m.predict(future)
        pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        pred.reset_index(inplace = True)
        test_data.reset_index(inplace = True)
        forecast = pd.concat([test_data, pred], axis=1)
        predicted_df = pd.concat([predicted_df,forecast], axis = 0)

C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\1119816113.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)
C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\1119816113.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fuel Price'] = train_data['Fuel Price']
16:30:44 - cmdstanpy - INFO - Chain [1] start processing
16:30:44 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\1119816113.py:7: S

In [22]:
folder = 'Modelling Output'
subfolder = 'PROPHET'
filename = 'predictions_w_holidays.csv'
write_path = os.path.join(directory,folder,subfolder,filename)
predicted_df.to_csv(write_path)

In [23]:
#Building Prophet Model w Holidays & Regressor (Price)
predicted_df = pd.DataFrame()
for i in range(0,4):
    for x in material_list:
        train_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] <= train_date[i])]
        df = train_data[['date','Sales Volumes in L15']]
        df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)

        #Adding holidays
        holiday_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Holiday Flag'] == 1)]['date'].unique()
        lockdown_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['LockDown Flag'] == 1)]['date'].unique()
        loyalty_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Loyalty Flag'] == 1)]['date'].unique()
        Marketing_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Marketing Flag'] == 1)]['date'].unique()
    
        public_holidays = pd.DataFrame({
        'holiday': 'public holidays',
        'ds': holiday_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        lockdows = pd.DataFrame({
        'holiday': 'lockdowns',
        'ds': lockdown_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        loyalty = pd.DataFrame({
        'holiday': 'loyalty',
        'ds': loyalty_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        marketing = pd.DataFrame({
        'holiday': 'loyalty',
        'ds': Marketing_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        holidays = pd.concat((public_holidays, lockdows,loyalty,marketing))
    
        #Adding Regressors (Price)
        df['Fuel Price'] = train_data['Fuel Price']
        test_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] >= test_date[i]) & (sales_data['date'] <= end_date[i])]
        m = Prophet(holidays=holidays)
        m.add_regressor('Fuel Price')
        m.fit(df)
        future = m.make_future_dataframe(periods=periods[i])
        future = future[(future['ds'] >= test_date[i]) & (future['ds'] <= end_date[i])]
        future = pd.merge(future, test_data[['date','Fuel Price']], left_on = 'ds', right_on = 'date', how= 'left')
        future.drop('date', inplace = True, axis = 1)
        forecast = m.predict(future)
        pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        pred.reset_index(inplace = True)
        test_data.reset_index(inplace = True)
        forecast = pd.concat([test_data, pred], axis=1)
        predicted_df = pd.concat([predicted_df,forecast], axis = 0)

C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\2012164997.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)
C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\2012164997.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fuel Price'] = train_data['Fuel Price']
16:31:42 - cmdstanpy - INFO - Chain [1] start processing
16:31:43 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\2012164997.py:7: S

In [24]:
folder = 'Modelling Output'
subfolder = 'PROPHET'
filename = 'predictions_w_price.csv'
write_path = os.path.join(directory,folder,subfolder,filename)
predicted_df.to_csv(write_path)

In [25]:
#Building Prophet Model w Holidays & Regressor (Price Change)
predicted_df = pd.DataFrame()
for i in range(0,4):
    for x in material_list:
        train_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] <= train_date[i])]
        df = train_data[['date','Sales Volumes in L15']]
        df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)

        #Adding holidays
        holiday_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Holiday Flag'] == 1)]['date'].unique()
        lockdown_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['LockDown Flag'] == 1)]['date'].unique()
        loyalty_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Loyalty Flag'] == 1)]['date'].unique()
        Marketing_list = sales_data[(sales_data['Material Number'] == x) & (sales_data['Marketing Flag'] == 1)]['date'].unique()
    
        public_holidays = pd.DataFrame({
        'holiday': 'public holidays',
        'ds': holiday_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        lockdows = pd.DataFrame({
        'holiday': 'lockdowns',
        'ds': lockdown_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        loyalty = pd.DataFrame({
        'holiday': 'loyalty',
        'ds': loyalty_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        marketing = pd.DataFrame({
        'holiday': 'loyalty',
        'ds': Marketing_list,
        'lower_window': 0,
        'upper_window': 1,
        })
        holidays = pd.concat((public_holidays, lockdows,loyalty,marketing))
    
        #Adding Regressors (Price)
        df['Fuel Price PCT Change'] = train_data['Fuel Price PCT Change']
        test_data = sales_data[(sales_data['Material Number'] == x) & (sales_data['date'] >= test_date[i]) & (sales_data['date'] <= end_date[i])]
        m = Prophet(holidays=holidays)
        m.add_regressor('Fuel Price PCT Change')
        m.fit(df)
        future = m.make_future_dataframe(periods=periods[i])
        future = future[(future['ds'] >= test_date[i]) & (future['ds'] <= end_date[i])]
        future = pd.merge(future, test_data[['date','Fuel Price PCT Change']], left_on = 'ds', right_on = 'date', how= 'left')
        future.drop('date', inplace = True, axis = 1)
        forecast = m.predict(future)
        pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        pred.reset_index(inplace = True)
        test_data.reset_index(inplace = True)
        forecast = pd.concat([test_data, pred], axis=1)
        predicted_df = pd.concat([predicted_df,forecast], axis = 0)

C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\3990706565.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename({'date': 'ds', 'Sales Volumes in L15': 'y'}, axis='columns', inplace = True)
C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12388\3990706565.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fuel Price PCT Change'] = train_data['Fuel Price PCT Change']
16:32:53 - cmdstanpy - INFO - Chain [1] start processing
16:32:53 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Shraddha.Mishra\AppData\Local\Temp\ipykernel_12

In [26]:
folder = 'Modelling Output'
subfolder = 'PROPHET'
filename = 'predictions_w_price_change.csv'
write_path = os.path.join(directory,folder,subfolder,filename)
predicted_df.to_csv(write_path)